<a href="https://colab.research.google.com/github/sf-rahul/Tensorflow/blob/master/charRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # Colab only.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
2.2.0-rc3


In [0]:
import numpy as np
import pandas as pd

In [0]:
#keras as ks.
ks = tf.keras

In [4]:
shakes_url = "https://homl.info/shakespeare"
filepath = ks.utils.get_file("shakespearre.txt",shakes_url)
with open(filepath) as f:
  shakes_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [5]:
len(shakes_text)

1115394

In [0]:
tokenizer = ks.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakes_text])

In [7]:
tokenizer.texts_to_sequences(["First"])


[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [0]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [10]:
print('no of distinct charcters:', max_id)
print('no of characters in dataset:' , dataset_size)

no of distinct charcters: 39
no of characters in dataset: 1


In [0]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakes_text]))-1

In [12]:
len(encoded)

1115394

In [0]:
train_size = len(encoded)*90//100

In [14]:
train_size


1003854

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [16]:
for i in dataset.take(5):
  print(i.numpy())

19
5
8
7
2


In [0]:
n_steps = 100
window_length = n_steps + 1 #last one will be target
dataset = dataset.window(window_length,shift=1,drop_remainder=True) # dataset will be nested datasets 

In [0]:
dataset = dataset.flat_map(lambda window : window.batch(window_length)) #will convert dataset in tensors

In [19]:
for i in dataset.take(1):
  print(i)

tf.Tensor(
[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
 17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
 10 15  3 13  0], shape=(101,), dtype=int64)


In [0]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)



In [0]:
x = []
for i in dataset.take(1):
  x= i.numpy()

In [22]:
x

array([[ 5,  8,  0, ...,  6,  5, 20],
       [ 0, 18,  3, ...,  2, 11,  1],
       [ 6,  1,  0, ..., 11,  5, 25],
       ...,
       [21,  1, 11, ...,  0, 21,  3],
       [ 2, 28,  0, ..., 18,  3, 13],
       [ 5,  9, 12, ...,  0, 13, 22]])

In [0]:
dataset = dataset.map(lambda windows : (windows[:,:-1],windows[:,-1:]))

In [0]:
dataset = dataset.map(lambda X_batch,Y_batch :( tf.one_hot(X_batch,depth=max_id),Y_batch))

In [25]:
for i in dataset.take(1):
  print(i)

(<tf.Tensor: shape=(32, 100, 39), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.

In [0]:
dataset = dataset.prefetch(1)

In [27]:
model = ks.models.Sequential([ks.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
                                 ks.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
                                 ks.layers.TimeDistributed(ks.layers.Dense(max_id,activation="softmax"))])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 128)         64896     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 128)         99072     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 39)          5031      
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss="categorical_crossentropy",optimizer="adam")
history = model.fit(dataset,epochs=20)

Epoch 1/20


ValueError: ignored